In [2]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

from sklearn.preprocessing import LabelEncoder 
from scipy.sparse import coo_matrix

import pickle

In [3]:
games_df=pd.read_csv('steam-200k.csv',header=None)
games_df.columns=['user_id','title','action','hours','hz']

games_df=games_df[games_df['action']=='purchase']

In [4]:
le = LabelEncoder()
le.fit(games_df['title'])

games_df['title']=le.transform(games_df['title'])

In [21]:
le.transform(['kek'])

ValueError: y contains previously unseen labels: ['kek']

In [5]:
games_df.head()

,user_id,title,action,hours,hz
0,151603712,4364,purchase,1.0,0
2,151603712,1678,purchase,1.0,0
4,151603712,3997,purchase,1.0,0
6,151603712,1679,purchase,1.0,0
8,151603712,2475,purchase,1.0,0


In [6]:
games_df_sample=games_df.iloc[:1000]
games_df_pivot_sample=games_df_sample.pivot_table(columns=['title'],index=['user_id'],values=['hours'])

In [7]:
games_df_pivot=games_df.pivot_table(columns=['title'],index=['user_id'],values=['hours'])
games_df_pivot.fillna(value=0,inplace=True)

games_df_pivot.head()

hours                                               ...            \
title    0    1    2    3    4    5    6    7    8    9     ... 5145 5146   
user_id                                                     ...             
5250      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
76767     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
86540     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
103360    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
144736    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

                                                 
title   5147 5148 5149 5150 5151 5152 5153 5154  
user_id                                          
5250     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
76767    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
86540    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
103360   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
144736   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 5155 columns]

In [8]:
games_df_pivot_train=games_df_pivot.sample(frac=0.8)
games_df_pivot_test=games_df_pivot.loc[games_df_pivot.index.difference(games_df_pivot_train.index)]

print(f"""Full shape: {games_df_pivot.shape}
Train shape: {games_df_pivot_train.shape}
Test shape: {games_df_pivot_test.shape}""") 

games_df_pivot_train_sparse=coo_matrix(games_df_pivot_train.values)
games_df_pivot_test_sparse=coo_matrix(games_df_pivot_test.values)

Full shape: (12393, 5155)
Train shape: (9914, 5155)
Test shape: (2479, 5155)


In [12]:
model = LightFM(loss='warp',random_state=42)
model.fit(games_df_pivot_train_sparse, epochs=150, num_threads=2)

In [19]:
{"res":i for i in model.predict([3],[1])}

{'res': -3.4573824405670166}

In [15]:
print("Train precision: %.2f" % precision_at_k(model, games_df_pivot_train_sparse, k=5).mean())
# print("Test precision: %.2f" % precision_at_k(model, games_df_pivot_test_sparse, k=5).mean())

Train precision: 0.31


In [12]:
pickle.dump(model,open('model.pickle','wb'))

In [17]:
%%writefile model.py

import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

from sklearn.preprocessing import LabelEncoder 
from scipy.sparse import coo_matrix

games_df=pd.read_csv('steam-200k.csv',header=None)
games_df.columns=['user_id','title','action','hours','hz']

games_df=games_df[games_df['action']=='purchase']

le = LabelEncoder()
le.fit(games_df['title'])

games_df['title']=le.transform(games_df['title'])

games_df_pivot=games_df.pivot_table(columns=['title'],index=['user_id'],values=['hours'])
games_df_pivot.fillna(value=0,inplace=True)

games_df_pivot_train=games_df_pivot.sample(frac=0.8)
games_df_pivot_test=games_df_pivot.loc[games_df_pivot.index.difference(games_df_pivot_train.index)]

games_df_pivot_train_sparse=coo_matrix(games_df_pivot_train.values)
games_df_pivot_test_sparse=coo_matrix(games_df_pivot_test.values)

model = LightFM(loss='warp',random_state=42)
model.fit(games_df_pivot_train_sparse, epochs=150, num_threads=2)

return model.predict([3],[1])

# print("Train precision: %.2f" % precision_at_k(model, games_df_pivot_train_sparse, k=5).mean())
# print("Test precision: %.2f" % precision_at_k(model, games_df_pivot_test_sparse, k=5).mean())

# pickle.dump(model,open('model.pickle','wb'))


Writing model.py
